#### Unnecessary - Copies just Repo records from one database to another
##### It IS a good example of batch queries and inserts

In [ ]:
import json

import sys
import mariadb
import time
import hashlib
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from kitchen_sink_class import load_single_line_from_file as load_it
pwd = load_it('./db.pwd')

this_is_necessary = False

if this_is_necessary:
    mydb = mariadb.connect(
        port=3306,
        host='127.0.0.1',
        user='w3hacknet',
        password=pwd,
        database='w3hacknet',
        autocommit=True
    )
    dest_db = mariadb.connect(
        port=13306,
        host='127.0.0.1',
        user='w3hacknet',
        password=pwd,
        database='w3hacknet',
        autocommit=False
    )

    cursor = mydb.cursor()
    dest_cursor = dest_db.cursor()
    cursor.execute('select owner, name, commit_count, min_date, max_date from repo;')
    running = True
    rec_count = 0
    while running:
        rslt_set = cursor.fetchmany(size=100)
        if rslt_set:
            batch = []
            for (owner, name, commit_count, min_date, max_date) in rslt_set:
                batch.append((owner, name, commit_count, min_date, max_date))
            dest_cursor.executemany('insert ignore into w3hacknet.repo (owner, name, commit_count, min_date, max_date) values (?, ?, ?, ?, ?)', batch)
            dest_db.commit()
            rec_count += len(batch)
            print(rec_count,'records copied')
        else:
            running = False
    cursor.close()
    dest_cursor.close()

#### Better way to clone a database, procedures included
##### On the From database command line:
`mysqldump -p w3hacknet --routines > dump_of_matts_db.sql`
##### On the To database command line:
`mysql -p w3hacknet <dump_of_matts_db.sql`

In [10]:
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from db_dependent_class import DBDependent
class E(DBDependent):
    def __init__(self):
        super().__init__()
    def main(self):
        single = ('sapi69','cabal-server','2011-10-28T12:43:15','UTC','7a49dd00afdd43b55fcfa64c225fae48cada58b6','111f9be771e9637e40f1547ee60b8f52')
        multi = [('sapi69','cabal-server','2011-10-28T12:43:15','UTC','7a49dd00afdd43b55fcfa64c225fae48cada58b6','111f9be771e9637e40f1547ee60b8f52'),
                 ('sapi69','cabal-server','2011-10-28T12:11:11','UTC','9a6f190f5421d7e3c5c360bc49c7e9974e0ed448','111f9be771e9637e40f1547ee60b8f52'),
                 ('sapi69','cabal-server','2011-10-19T12:29:48','UTC','eb54f1d0f240c624a009a5a81d45c80210c22ccc','111f9be771e9637e40f1547ee60b8f52')
                 ]
        self.get_cursor().execute(' CALL w3hacknet.addUpdateRepo(?,?,?,?,?,?)', single)
        print('Executed single')
        self.get_cursor().executemany(' CALL w3hacknet.addUpdateRepo(?,?,?,?,?,?)', multi)
        print('Completed')

if __name__ == "__main__":
    print('Running!')
    E().main()

Running!
Executed single


OperationalError: This command is not supported in the prepared statement protocol yet

#### Fiddle with String join function
##### Claims to be faster than "str"+"other Str"

In [15]:
print(''.join(('one', str(2), 'three')))
print('one'.join((str(3), 'middle?', '5ive', 'end')))

one2three
3onemiddle?one5iveoneend


In [19]:
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from db_dependent_class import DBDependent
class E(DBDependent):
    def __init__(self):
        super().__init__()
    def main(self):
        self.get_cursor().callproc('ReserveNextRepoForEvaluation', ['matt'])
        one = self.cursor.fetchone()
        print(one)
        print(one.__class__)
        print(one[0], one[1])

if __name__ == "__main__":
    print('Running!')
    E().main()

Running!
('sebastiandres', 'git-drawing')
<class 'tuple'>
sebastiandres git-drawing


#### Semi-recursive JSON parser (e.g. "level1.level2.desired_value")

In [28]:
local_lib_dir = '../../../python/lib/'
sys.path.append(local_lib_dir)
from git_hub_client import fetch_json_value


with open('./sample.json', 'r') as j:
    sample = json.load(j)
print(fetch_json_value('language', sample))
print(fetch_json_value('license.key', sample))
print(fetch_json_value('organization.login', sample))
print(fetch_json_value('organization.weird.other', sample))
print(fetch_json_value('not_there', sample))
print(fetch_json_value('bad.shit.other', sample))


Jupyter Notebook
gpl-3.0
Deadman-DAO
True


StopIteration: key not_there not found.

#### Testing EPOCH timestamp from GitHub

In [30]:
from datetime import datetime as datingdays
datingdays.fromtimestamp(1653782400)

datetime.datetime(2022, 5, 28, 17, 0)